In [1]:
import pandas as pd
import langid
from tqdm import tqdm
import text2emotion as te
tqdm.pandas()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ben\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ben\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ben\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
bit=pd.read_csv(r"C:\Users\Ben\Desktop\Diplomatiki\Reddit 101\Trying PRAW\btc_bitcoin_2021_uptoAug.csv")

In [ ]:
bit.selftext=bit.body.astype(str)
bit.selftext=bit.body.fillna('NO TEXT')
#bit=bit[bit.selftext!='nan']

#english
bit['lang']=bit.title.progress_apply(langid.classify)


 44%|████████████████████████████████▉                                          | 27260/62174 [01:32<01:59, 292.76it/s]

In [ ]:
bit['lang_text']=bit.body.progress_apply(langid.classify)


In [ ]:
bit.lang=bit.lang.apply(lambda x: x[0])
bit=bit[bit.lang=='en']

bit.loc[bit.body=='nan','text']=bit.title
bit.loc[bit.body!='nan','text']=bit.selftext

# Train model

In [ ]:
# utilities
import re
import numpy as np
import pandas as pd
import string
# plotting
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# nltk
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
# sklearn
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report

wn = nltk.WordNetLemmatizer()
ps = nltk.PorterStemmer()
cached_stopwords=stopwords.words('english')

remove_hashtags=lambda x: ' '.join(re.sub("(#[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x).split())

# removes hashtags mentions links and puncuation! 
regex_magic= lambda x: ' '.join(re.sub("(#[A-Za-z0-9]+)|(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x).split())
regex_notso_magic= lambda x: ' '.join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x).split())
#TEXT CLEANING FUNCTION
def clean_text(text):
    text=regex_magic(text).lower()
    text= re.sub('[0-9]+', '', text)#removed numbers
    tokens = re.split('\W+', text)    # tokenization
    text = [ps.stem(word) for word in tokens if word not in cached_stopwords]  # remove stopwords and stemming
    text = [wn.lemmatize(word) for word in text]
    return text

#functions for printing the results of the Model
def model_Evaluate(model):
    
    # Predict values for Test dataset
    y_pred = model.predict(X_test)

    # Print the evaluation metrics for the dataset.
    print(classification_report(y_test, y_pred))
    
    # Compute and plot the Confusion matrix
    cf_matrix = confusion_matrix(y_test, y_pred)

    categories  = ['Negative','Positive']
    group_names = ['True Neg','False Pos', 'False Neg','True Pos']
    group_percentages = ['{0:.2%}'.format(value) for value in cf_matrix.flatten() / np.sum(cf_matrix)]

    labels = [f'{v1}\n{v2}' for v1, v2 in zip(group_names,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)

    sns.heatmap(cf_matrix, annot = labels, cmap = 'Blues',fmt = '',
                xticklabels = categories, yticklabels = categories)

    plt.xlabel("Predicted values", fontdict = {'size':14}, labelpad = 10)
    plt.ylabel("Actual values"   , fontdict = {'size':14}, labelpad = 10)
    plt.title ("Confusion Matrix", fontdict = {'size':18}, pad = 20)

In [ ]:
# Importing the dataset
sent140 = pd.read_csv(r"D:\DTU\Semester 3 - Fall 2020\Social Graphs and Interactions\__Final Project__\Data Proofs\trainingandtestdata\sentiment140.csv",
                      encoding="ISO-8859-1" , names=["sentiment", "ids", "date", "flag", "user", "text"])
#taking the columns that we need
sent140 = sent140[['sentiment','text']]
sent140['sentiment'] = sent140['sentiment'].replace(4,1)
#preprocessing the text
sent140['text_clean']=sent140.text.progress_apply(clean_text)
sent140['text_clean_join']=sent140.text_clean.progress_apply(lambda x: ' '.join(x))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(sent140['text_clean_join'],
                                                    sent140.sentiment,
                                                    test_size = 0.05, random_state = 42)
word2vec = TfidfVectorizer(ngram_range=(1,2), max_features=50000)
word2vec.fit(X_train)
X_train = word2vec.transform(X_train)
X_test  = word2vec.transform(X_test)

In [ ]:
#Linear SVC
clf = LogisticRegression(C=2,n_jobs=-1,max_iter=10000)
#Train the model
clf.fit(X_train, y_train)
model_Evaluate(clf)

In [ ]:
# Getting the sentiment of the Tweets
def get_sentiment(text):
    text=word2vec.transform([text])
    neg,pos=clf.predict_proba(text)[0]
    if neg>.6:
        sentiment='Negative'
    elif pos>.6:
        sentiment='Positive'
    else:
        sentiment='Netural'
    return neg,pos,sentiment

list2doc=lambda x: ' '.join(x)

In [ ]:
bit['text_processed']=bit['text'].progress_apply(lambda x: list2doc(clean_text(x)))
bit['sentiment_scores']=bit.text_processed.progress_apply(get_sentiment)

In [ ]:
bit['Date']=pd.to_datetime(bit.date) 
bit=bit.set_index('Date')

In [ ]:
bit.sample(10).T

In [ ]:
btcp=pd.read_csv(r"C:\Users\Ben\Desktop\Diplomatiki\Datasets\BTC_USD_2020-09-13_2021-09-12-CoinDesk.csv")
btcp['Date_index']=pd.to_datetime(btcp.Date)
btcp=btcp.set_index('Date_index')

btcp=btcp['2021':'2021-08']

In [ ]:
d1=bit.resample('d').sum()[['polarity']]

In [ ]:
df=pd.concat([d1,btcp[['Closing Price (USD)']]],axis=1)
df=df[df.polarity.notna()]
df=df.rename(columns={'Closing Price (USD)':'btc_close'})
df['btc_close_diff']=df.btc_close.diff()

In [ ]:
a1=df.resample('w').sum().btc_close_diff/60
a2=df.resample('w').sum().polarity
pd.concat([a1,a2],axis=1).plot(figsize=(15,8));